# Week 6 Assignment

## Instructions

Inputs (found inside /home/rbif/week6/):
dgorgon_reference.fa - The wildtype reference sequence of D.gorgon
harrington_clinical_data.txt - The clinical data of the samples including a unique patient name, the color of the mold, and a sequence barcode*.
hawkins_pooled_sequences.fastq - The fastq data that contains the sequences of all the samples pooled together.
*The barcode is the first 5 basepairs of a sequence. It is a unique identifier that determines which sample that read belongs to. So if Tim's barcode was ATCCC then every sequence that begins with ATCCC is a read assigned to Tim.

There is a visual representation attached: Visual_pipeline_representation.pdf
Steps. Anything highlighted in yellow means I will provide you a python function to use and incorporate into your script. 
You will not be able to use functions from BioPython for this assignment.
1. Demultiplex the pooled fastq - Create a single python script that reads in the pooled fastq* and outputs 50 new fastq files that contain only sequences belonging to that sample. This means using the barcode from the clinical data, finding the match at the start of a given read in the pooled fastq, and writing the read to its respective file. It should at the same time perform the steps 1A and 1B below. 
*script is /home/rbif/week6/necessary_scripts/parseFastq.py You just need to copy the (class) function into your own script and use the function to help you trim.*
1A. Trim the beginning of the reads - Write a python function that removes the barcode and it's associated quality scores from the read.
1B. Trim the ends of the reads - Write a python function that removes the ends of the reads based on the following condition: The tail ends of the reads have degraded in quality and must be trimmed before being mapped. If the end of the read has at least two consecutive quality scores of D or F, then remove the rest of the read.


Example:
From
@arbitrary-seq-111
ATCGATGCACCCC
+
IIIDIIIIIDDFF 
To


@arbitrary-seq-111
ATCGATGCA
+
IIIDIIIII
**This is any combination of D and/or F**
Everything in red would be trimmed off
IIIIIFDFF
Or
IIFFDII
Or
IIFFIIII

Final output of Step 1: 50 fastq files in a folder called fastqs. Each fastq inside of that directory should have the following format:
{name}_trimmed.fastq (ie tim_trimmed.fastq)

2. Perform alignment on each FASTQ to the reference sequence. Write a python function that calls the bwa command. This will transform the FASTQs to samfiles.
*Hint* Use the python packages os or subprocess to call a command line tool. 
*Note* In order to use bwa, the reference file must first be indexed. This can be accomplished with bwa index ref.fa
Then you can use bwa mem ref.fa {name}_trimmed.fastq > {name}.sam
to perform the alignment. This is taken from http://bio-bwa.sourceforge.net/bwa.shtml
The mapping_to_reference.pdf shows reads that are aligning to the reference. Bwa tries to find the best match for each individual read and maps it to a known reference.
3. Convert the samfiles to bamfiles - Use python to wrap the below commands:
Use samtools view to convert the SAM file into a BAM file. BAM is the binary format corresponding to the SAM text format. Run:
samtools view -bS {name}.sam > {name}.bam
Use samtools sort to convert the BAM file to a sorted BAM file.
samtools sort {name}.bam {name}.sorted
samtools sort -o {name}.sorted.bam {name}.bam

Then we need to index the sorted bam.
samtools index {name}.sorted.bam 
We now have a sorted BAM file called {name}.sorted.bam. Sorted BAM is a useful format because the alignments are (a) compressed, which is convenient for long-term storage, and (b) sorted, which is convenient for variant discovery. 
Taken from: http://bowtie-bio.sourceforge.net/bowtie2/manual.shtml#getting-started-with-bowtie-2-lambda-phage-example
Once the sorted.bam files are created, remove the prior .sam files and the .bam files. 
4. Variant Discovery - Use the python pysam pileup function to discover variants in each sorted bam file. 
The script is located /home/rbif/week6/necessary_scripts/getMutations.py
Copy the function, pileup, to your own script. This needs to be modified. Details on where to modify it are inside the script. 
You are using this script to find what position along the reference has a mutation. 
In this case we are looking for a SNP (single nucleotide polymorphism). If a position contains DNA basepairs other than the wildtype, then we can consider it as a SNP and should be reported.
**You can consider the reference strain to be wildtype **

5. Create a report - Using python, create a report that outputs what nucleotide position and mutation is responsible for each color of the mold. Also print out the number of sequences that were used for each sample. 
Example:
The green mold was caused by a mutation in position 23. The wildtype base was A and the mutation was C.
The black mold…

Sample Tim had a green mold, 320 reads, and had 32% of the reads at position 23 had the mutation C. 
Sample Kristen ...

Final Deliverables.
A single python script with comments that performs all of the above steps called pipeline.py
A folder called fastqs that contains the demultiplexed fastqs for each sample.
A folder called bams that contains the sorted.bam file for each sample.
A text file called report.txt that looks like step 5.
A readme that tells the user how to use the script.

## Workflow outline

### Demultiplex the pooled fastq by barcode

## Environment Review

In [1]:
import sys

sys.version

'3.6.10 |Anaconda, Inc.| (default, Mar 25 2020, 23:51:54) \n[GCC 7.3.0]'

In [4]:
%%bash

ls
echo ""
cd necessary_scripts/
ls
echo ""
echo "----------------------"
echo "Line Count of getMutations"
wc -l getMutations.py
echo ""
echo "----------------------"
echo "Line Count of parseFastq.py"
wc -l parseFastq.py

dgorgon_reference.fa
harrington_clinical_data.txt
hawkins_pooled_sequences.fastq
necessary_scripts
Week6_Lay_that_Pipe.ipynb

faker.sorted.bam
faker.sorted.bam.bai
getMutations.py
parseFastq.py

----------------------
Line Count of getMutations
29 getMutations.py

----------------------
Line Count of parseFastq.py
92 parseFastq.py


### Initial Thoughts

These files are just too big, to look cat them out above, let's bring these files into jupyter

### hawking_pooled_sequences.fastq

In [7]:
%%bash

head -10 hawkins_pooled_sequences.fastq

@seq13534-419
GCAGTAGCGGTCATAAGTGGTACATTACGAGATTCGGAGTACCATAGATTCGCATGAATCCCTGTGGATACGAGAGTGTGAGATATATGTACGCCAATCCAGTGTGATACCCATGAGATTTAGGACCGATGATGGTTGAGGACCAAGGATTGACCCGATGGATGCAGATTTGACCCCAGATAGAATAAATGCGATGAGATGATTTGGCCGATAGATAGATAGTGTCGTGAGGTGACGTCCGTCACTGGACGAA
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIDIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIFFFFDFFDFFDDFDFDFFFFDDFFDDFDDFF
@seq86249-867
GGATTAGCGGTCATAAGTCGTACATTACGAGATTCGGAGTACCATAGATTCGCATGAATCCCTGTGGATACGAGAGTGTGAGATATATGTACGCCAATCCAGTGTGATACCCATGAGATTTAGGACCGATGATGGTTGAGGACCAAGGATTGACCCGATGGATGCAGATTTGACCCCAGATAGAATAAATGCGATGAGATGATTTGGCCGATAGATAGATAGAGGTCAGTATAACCTCTCAAAGCTTTATCTACGGATGGATCCGCGC
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIDIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

### getMutations.py

pysam needs python 3.6 downgrading from 3.8 now

In [2]:
import pysam

#This is a slightly modified version from here: https://pysam.readthedocs.io/en/latest/api.html
# What is a pileup? It is "piling up" all the reads that have mapped to a given position of your reference.
# It is a useful way to see what reads have a mutation and what don't. 

def pileup():
    #test file, replaced with the sorted.bam you are using. Make sure it is indexed! 
    #(Use samtools index yourbam.sorted.bam)
    samfile = pysam.AlignmentFile("/home/rbif/week6/necessary_scripts/faker.sorted.bam", "rb")

    #Since our reference only has a single sequence, we're going to pile up ALL of the reads. 
    #Usually you would do it in a specific region (such as chromosome 1, position 1023 to 1050 for example)
    for pileupcolumn in samfile.pileup():
        print ("coverage at base %s = %s" % (pileupcolumn.pos, pileupcolumn.n))
        #use a dictionary to count up the bases at each position
        ntdict = {}
        for pileupread in pileupcolumn.pileups:
            if not pileupread.is_del and not pileupread.is_refskip:
                # You can uncomment the below line to see what is happening in the pileup. 
                # print ('\tbase in read %s = %s' % (pileupread.alignment.query_name, pileupread.alignment.query_sequence[pileupread.query_position]))
                base = pileupread.alignment.query_sequence[pileupread.query_position]
                
                ########## ADD ADDITIONAL CODE HERE ############# 
                # Populate the ntdict with the counts of each base 
                # This dictionary will hold all of the base read counts per nucletoide per position.
                # Use the dictionary to calculate the frequency of each site, and report it if if the frequency is NOT  100% / 0%. 
                #############################################
        print (ntdict)
    samfile.close()

if __name__=="__main__":
    pileup()

coverage at base 0 = 980
{}
coverage at base 1 = 980
{}
coverage at base 2 = 980
{}
coverage at base 3 = 980
{}
coverage at base 4 = 980
{}
coverage at base 5 = 980
{}
coverage at base 6 = 980
{}
coverage at base 7 = 980
{}
coverage at base 8 = 980
{}
coverage at base 9 = 980
{}
coverage at base 10 = 980
{}
coverage at base 11 = 980
{}
coverage at base 12 = 980
{}
coverage at base 13 = 980
{}
coverage at base 14 = 980
{}
coverage at base 15 = 980
{}
coverage at base 16 = 980
{}
coverage at base 17 = 980
{}
coverage at base 18 = 980
{}
coverage at base 19 = 980
{}
coverage at base 20 = 980
{}
coverage at base 21 = 980
{}
coverage at base 22 = 980
{}
coverage at base 23 = 980
{}
coverage at base 24 = 980
{}
coverage at base 25 = 980
{}
coverage at base 26 = 980
{}
coverage at base 27 = 980
{}
coverage at base 28 = 980
{}
coverage at base 29 = 980
{}
coverage at base 30 = 980
{}
coverage at base 31 = 980
{}
coverage at base 32 = 980
{}
coverage at base 33 = 980
{}
coverage at base 34 = 98

### parseFastq.py

In [3]:
import argparse
#Example use is 
# python parseFastq.py --fastq /home/rbif/week6/hawkins_pooled_sequences.fastq


################################################
# You can use this code and put it in your own script
class ParseFastQ(object):
    """Returns a read-by-read fastQ parser analogous to file.readline()"""
    def __init__(self,filePath,headerSymbols=['@','+']):
        """Returns a read-by-read fastQ parser analogous to file.readline().
        Exmpl: parser.next()
        -OR-
        Its an iterator so you can do:
        for rec in parser:
            ... do something with rec ...
 
        rec is tuple: (seqHeader,seqStr,qualHeader,qualStr)
        """
        if filePath.endswith('.gz'):
            self._file = gzip.open(filePath)
        else:
            self._file = open(filePath, 'rU')
        self._currentLineNumber = 0
        self._hdSyms = headerSymbols
         
    def __iter__(self):
        return self
     
    def next(self):
        """Reads in next element, parses, and does minimal verification.
        Returns: tuple: (seqHeader,seqStr,qualHeader,qualStr)"""
        # ++++ Get Next Four Lines ++++
        elemList = []
        for i in range(4):
            line = self._file.readline()
            self._currentLineNumber += 1 ## increment file position
            if line:
                elemList.append(line.strip('\n'))
            else: 
                elemList.append(None)
         
        # ++++ Check Lines For Expected Form ++++
        trues = [bool(x) for x in elemList].count(True)
        nones = elemList.count(None)
        # -- Check for acceptable end of file --
        if nones == 4:
            raise StopIteration
        # -- Make sure we got 4 full lines of data --
        assert trues == 4,\
               "** ERROR: It looks like I encountered a premature EOF or empty line.\n\
               Please check FastQ file near line number %s (plus or minus ~4 lines) and try again**" % (self._currentLineNumber)
        # -- Make sure we are in the correct "register" --
        assert elemList[0].startswith(self._hdSyms[0]),\
               "** ERROR: The 1st line in fastq element does not start with '%s'.\n\
               Please check FastQ file near line number %s (plus or minus ~4 lines) and try again**" % (self._hdSyms[0],self._currentLineNumber) 
        assert elemList[2].startswith(self._hdSyms[1]),\
               "** ERROR: The 3rd line in fastq element does not start with '%s'.\n\
               Please check FastQ file near line number %s (plus or minus ~4 lines) and try again**" % (self._hdSyms[1],self._currentLineNumber) 
        # -- Make sure the seq line and qual line have equal lengths --
        assert len(elemList[1]) == len(elemList[3]), "** ERROR: The length of Sequence data and Quality data of the last record aren't equal.\n\
               Please check FastQ file near line number %s (plus or minus ~4 lines) and try again**" % (self._currentLineNumber) 
         
        # ++++ Return fatsQ data as tuple ++++
        return tuple(elemList)
##########################################################################



if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("-f", "--fastq", required=True, help="Place fastq inside here")
    args = parser.parse_args()

    #This is an example of how to use the function in your own code
    
    fastqfile = ParseFastQ(args.fastq)

    #A fastq read contains 4 lines
    for fastq_obj in fastqfile:
        #This fastq_obj is a tuple that has length of 4 and corresponds to those 4 lines
        #This is the header
        print(fastq_obj[0])
        #This is the sequence
        print(fastq_obj[1])
        #This is the separator
        print(fastq_obj[2])
        #This is the quality score
        print(fastq_obj[3])
        
        #Just an indicator showing the fastq "blocks"
        print('*'*10 + '==='*10 + '*' * 10)

/home/ruggm/anaconda3/envs/learn/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: 'U' mode is deprecated


TypeError: iter() returned non-iterator of type 'ParseFastQ'

In [12]:
print("hello")

hello
